In [1]:
import numpy as np
import pandas as pd
import ta
import talib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [31]:
symbols = ['EURUSD','GBPUSDmicro','USDCHFmicro','USDJPYmicro','USDCADmicro']

timeframe = 'D1'

df = pd.read_csv("data/"+ symbols[0]+ timeframe+ ".csv")
df = df[['Date','Open','High','Low','Close']]
df['Date'] = pd.to_datetime(df['Date'])

df['High1'] = df['High'].shift(1)
df['High2'] = df['High'].shift(2)
df['High3'] = df['High'].shift(3)

df['Low1'] = df['Low'].shift(1)
df['Low2'] = df['Low'].shift(2)
df['Low3'] = df['Low'].shift(3)

df['Close1'] = df['Close'].shift(1)
df['Close2'] = df['Close'].shift(2)
df['Close3'] = df['Close'].shift(3)


df.dropna(inplace=True)

df.set_index ('Date', inplace = True)

df

,Open,High,Low,Close,High1,High2,High3,Low1,Low2,Low3,Close1,Close2,Close3
Date,,,,,,,,,,,,,
2013-01-04,1.30364,1.30884,1.29975,1.30677,1.31889,1.32992,1.32192,1.30246,1.31567,1.31977,1.30366,1.31888,1.32053
2013-01-07,1.30779,1.31276,1.30165,1.31252,1.30884,1.31889,1.32992,1.29975,1.30246,1.31567,1.30677,1.30366,1.31888
2013-01-08,1.31253,1.31388,1.30569,1.30802,1.31276,1.30884,1.31889,1.30165,1.29975,1.30246,1.31252,1.30677,1.30366
2013-01-09,1.30804,1.30953,1.30364,1.30540,1.31388,1.31276,1.30884,1.30569,1.30165,1.29975,1.30802,1.31252,1.30677
2013-01-10,1.30538,1.32789,1.30383,1.32668,1.30953,1.31388,1.31276,1.30364,1.30569,1.30165,1.30540,1.30802,1.31252
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,1.10887,1.10940,1.10692,1.10863,1.10960,1.11248,1.11443,1.10700,1.10663,1.11072,1.10895,1.10764,1.11202
2019-12-26,1.10875,1.11090,1.10824,1.10969,1.10940,1.10960,1.11248,1.10692,1.10700,1.10663,1.10863,1.10895,1.10764
2019-12-27,1.10975,1.11884,1.10943,1.11763,1.11090,1.10940,1.10960,1.10824,1.10692,1.10700,1.10969,1.10863,1.10895


In [33]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG


class PFR(Strategy):
    def init(self):
        price = self.data.Close
        
    def next(self):
        price = self.data.High
        Min = min(self.data.Low1, self.data.Low2)
        
        if self.data.Low < Min  and self.data.Close > self.data.Close1 :
            amplitude = abs(self.data.High - self.data.Low)
            sl = price - (1 * amplitude)
            tp = price + (1.618 * amplitude)
            self.buy(limit = price, sl = sl, tp = tp)

bt = Backtest(df, PFR, commission=.002, exclusive_orders=True)
stats = bt.run()
print(stats)
bt.plot(resample= False)


C:\Users\juini\anaconda3\envs\py38\lib\site-packages\backtesting\backtesting.py:899: UserWarning: A SL/TP order would execute in the same bar as its contingent upon stop/limit order. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(


Start                     2013-01-04 00:00:00
End                       2019-12-31 00:00:00
Duration                   2552 days 00:00:00
Exposure Time [%]                     47.5166
Equity Final [$]                      6710.82
Equity Peak [$]                       10175.6
Return [%]                           -32.8918
Buy & Hold Return [%]                 14.1142
Max. Drawdown [%]                    -34.7348
Avg. Drawdown [%]                    -8.02429
Max. Drawdown Duration     2100 days 00:00:00
Avg. Drawdown Duration      501 days 00:00:00
# Trades                                  199
Win Rate [%]                          34.6734
Best Trade [%]                        2.18109
Worst Trade [%]                      -2.10919
Avg. Trade [%]                      -0.200248
Max. Trade Duration          41 days 00:00:00
Avg. Trade Duration           5 days 00:00:00
Profit Factor                        0.600196
Expectancy [%]                       0.786396
SQN                               